# Pandas RAG 모델 구축

**[필요한 라이브러리 호출 및 API키 설정]**

In [1]:
from dotenv import load_dotenv; load_dotenv()  # dev에서만
import os
OPENAI_API = os.getenv("OPENAI_API")

In [2]:
import os
os.chdir(r'C:\Users\Hopedom\Documents\DS5-LangChain\Langchain-RAG\\')

## **[문서 로드/분할 및 벡터 임베딩]**

### [문서를 LangChain Document 객체로 로드]

</br>

$$\text{DirectoryLoader} \xrightarrow{\text{탐색 및 경로 전달}} \text{UnstructuredFileLoader} \xrightarrow{\text{파싱 및 텍스트 추출}} \text{Document 객체}$$


- `DirectoryLoader`는 `pandas/doc/source 디렉터리` 내부를 탐색하며, 발견된 모든 `rst` 파일 경로를 `UnstructuredFileLoader`에게 전달하는 역할 수행
- `UnstructuredFileLoader`는 `DirectoryLoader`가 찾은 개별 rst 파일 경로를 받아, 파일 내용을 읽고 rst 마크업을 어느 정도 제거하여 순수한 텍스트를 추출하는 실질적인 파싱 작업을 수행

In [3]:
import os
from langchain_community.document_loaders import DirectoryLoader, UnstructuredFileLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from typing import List

# 📌 1. 로컬 .rst 파일 로드
# 로컬 Pandas 문서 소스 경로
PANDAS_DOC_PATH = "../pandas/doc/source"
print(f"로컬 경로 '{PANDAS_DOC_PATH}'에서 .rst 파일 로드 시작...")

# DirectoryLoader: 지정된 경로에서 .rst 파일을 찾고 UnstructuredFileLoader로 로드합니다.
# .rst 파일은 일반 텍스트 파일이므로 정확한 파싱을 위해 UnstructuredFileLoader를 사용합니다.
loader = DirectoryLoader(
    path=PANDAS_DOC_PATH,
    glob="**/*.rst",  # 재귀적으로 모든 .rst 파일 검색
    loader_cls=UnstructuredFileLoader,
    loader_kwargs={"autodetect_encoding": True},
    show_progress=True
)

# .rst 파일의 내용을 LangChain Document 객체로 로드
documents = loader.load()

print(f"로드된 Pandas 문서 객체 개수: {len(documents)}개")

로컬 경로 '../pandas/doc/source'에서 .rst 파일 로드 시작...


  9%|▉         | 19/211 [00:24<01:30,  2.12it/s][WARNING] Reference not found for 'np.r' at C:\Users\Hopedom\Documents\DS5-LangChain\Langchain-RAG\..\pandas\doc\source\getting_started\comparison\comparison_with_r.rst line 579 column 1

  9%|▉         | 20/211 [00:25<01:43,  1.84it/s][WARNING] Could not load include file includes/introduction.rst at C:\Users\Hopedom\Documents\DS5-LangChain\Langchain-RAG\..\pandas\doc\source\getting_started\comparison\comparison_with_sas.rst line 15 column 1
[WARNING] Could not load include file includes/copies.rst at C:\Users\Hopedom\Documents\DS5-LangChain\Langchain-RAG\..\pandas\doc\source\getting_started\comparison\comparison_with_sas.rst line 71 column 1
[WARNING] Could not load include file includes/construct_dataframe.rst at C:\Users\Hopedom\Documents\DS5-LangChain\Langchain-RAG\..\pandas\doc\source\getting_started\comparison\comparison_with_sas.rst line 94 column 1
[WARNING] Could not load include file includes/limit.rst at C:\Users\Hopedom\Docum

로드된 Pandas 문서 객체 개수: 211개


### [청크 분할 및 메타데이터 추가]

In [4]:
# PDF 파일 예시와 달리, 문서 원본이 rst이므로 chunk_overlap을 200으로 설정하여 문맥 보존 강화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200, 
    separators=["\n\n", "\n", " ", ""] # rst 마크업을 고려한 기본 분리자
)
chunks = text_splitter.split_documents(documents)

# RAG 멀티 도메인 필터링을 위해 'library': 'pandas' 메타데이터 추가
for chunk in chunks:
    # DirectoryLoader가 'source' 경로를 자동으로 추가해줍니다.
    chunk.metadata['library'] = 'pandas' 
    
print(f"분할된 최종 청크 개수: {len(chunks)}개")
print(f"첫 번째 청크의 메타데이터: {chunks[0].metadata}")

분할된 최종 청크 개수: 3449개
첫 번째 청크의 메타데이터: {'source': '..\\pandas\\doc\\source\\development\\community.rst', 'library': 'pandas'}


### [벡터 임베딩 및 ChromaDB 저장]

In [5]:
#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
print("\n벡터 임베딩 및 ChromaDB 저장 시작 (OpenAI 'text-embedding-3-small' 사용)...")

vectorstore = Chroma.from_documents(
    chunks, 
    OpenAIEmbeddings(model = 'text-embedding-3-small'),
    persist_directory='./chromadb/pandas_rst' 
)
retriever = vectorstore.as_retriever()

print("✅ Pandas RAG 데이터베이스 구축 완료.")
print(f"ChromaDB 저장 위치: './chromadb/pandas_rst'")


벡터 임베딩 및 ChromaDB 저장 시작 (OpenAI 'text-embedding-3-small' 사용)...
✅ Pandas RAG 데이터베이스 구축 완료.
ChromaDB 저장 위치: './chromadb/pandas_rst'


## **[프롬프트와 모델 선언]**

In [6]:
from langchain_core.prompts import load_prompt, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 로드 (LangChain Hub 또는 수동 정의)

try:
    # LangChain Hub에서 공식 RAG 프롬프트를 로드합니다.
    # rlm/rag-prompt 대신 'lc://prompts/rag-prompt/rag-prompt' 경로를 사용합니다.
    prompt = load_prompt("lc://prompts/rag-prompt/rag-prompt")
    
    # 로드된 프롬프트의 유형과 메시지 수 확인
    print("INFO: LangChain Hub 프롬프트가 'load_prompt'를 통해 성공적으로 로드되었습니다.")
    # print(f"프롬프트 유형: {type(prompt)}, 메시지 수: {len(prompt.messages)}")

except Exception as e:
    # load_prompt가 실패하거나 인터넷 연결 문제 등이 있을 경우를 대비한 대체 방법
    print(f"경고: LangChain Hub 프롬프트 로드에 실패했습니다. (오류: {e}) 프롬프트를 수동으로 정의합니다.")
    
    # RAG 프롬프트를 수동으로 정의 (rlm/rag-prompt의 일반적인 템플릿과 유사)
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an assistant for question-answering tasks. "
                "Use the following pieces of retrieved context to answer the question. "
                "If you don't know the answer, just say that you don't know. "
                "Use three sentences maximum and keep the answer concise.\n\n"
                "Context: {context}",
            ),
            ("human", "{question}"),
        ]
    )

경고: LangChain Hub 프롬프트 로드에 실패했습니다. (오류: Loading from the deprecated github-based Hub is no longer supported. Please use the new LangChain Hub at https://smith.langchain.com/hub instead.) 프롬프트를 수동으로 정의합니다.


In [7]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [8]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\nContext: {context}"), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]

In [9]:
# 언어 모델 (LLM) 선언

# 질문-답변 생성에 사용할 모델 선언
llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

print(f"✅ LLM 선언 완료: {llm.model_name}")

✅ LLM 선언 완료: gpt-5-nano


## **[Chain 구축]**

1. 질문을 받습니다.
2. retriever가 문서 조각(docs)을 검색합니다.
3. format_docs 함수가 docs를 단일 문자열 {context}로 만듭니다.
4. {context}와 {question}이 prompt 템플릿에 들어갑니다.
5. LLM이 답변을 생성하고, StrOutputParser가 이를 문자열로 변환합니다.

In [10]:
# Retriever로 검색한 유사 문서의 내용을 하나의 string으로 결합하는 함수 (Format Docs)
def format_docs(docs):
    """검색된 LangChain Document 객체들을 하나의 문자열 컨텍스트로 결합합니다."""
    return "\n\n".join(doc.page_content for doc in docs)

In [11]:
rag_chain = (
    {
        # context: retriever의 검색 결과를 format_docs 함수를 통해 문자열로 전달
        "context": retriever | format_docs, 
        # question: 원본 질문을 그대로 다음 단계로 전달
        "question": RunnablePassthrough()
    }
    | prompt  # 이전에 수동 정의된 prompt 객체 사용
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인(rag_chain) 구축 완료.")

✅ RAG 체인(rag_chain) 구축 완료.


In [12]:
rag_chain.get_graph().print_ascii()

             +---------------------------------+          
             | Parallel<context,question>Input |          
             +---------------------------------+          
                    ***                ***                
                 ***                      ***             
               **                            ***          
+----------------------+                        **        
| VectorStoreRetriever |                         *        
+----------------------+                         *        
            *                                    *        
            *                                    *        
            *                                    *        
    +-------------+                       +-------------+ 
    | format_docs |                       | Passthrough | 
    +-------------+*                      +-------------+ 
                    ***                ***                
                       ***          ***                 

## [RAG 질의 테스트]

In [13]:
question = "Pandas에서 누락된 값(Missing Values)을 확인하는 가장 일반적인 메서드는 무엇인가요?"
print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas에서 누락된 값(Missing Values)을 확인하는 가장 일반적인 메서드는 무엇인가요?

[답변]: 가장 일반적으로 사용하는 방법은 isna() 또는 isnull()로 누락 값을 확인하는 것입니다. 이 함수들은 DataFrame이나 Series에서 누락 값 위치를 True로 표시하는 불리언 시퀀스를 반환합니다.


In [14]:
question = "Pandas에서 datetime 형식의 열을 처리하는 방법은 무엇인가요?"
print(f"\n[질문]: {question}")

# RAG 체인을 통해 질문 실행
response = rag_chain.invoke(question)
print(f"\n[답변]: {response}")


[질문]: Pandas에서 datetime 형식의 열을 처리하는 방법은 무엇인가요?

[답변]: - 먼저 문자열인 열이라면 pd.to_datetime(열)으로 datetime64[ns, UTC]의 pandas.Timestamp 객체로 변환합니다. 
- 파일을 읽을 때는 parse_dates=[열]를 사용하면 자동으로 Timestamp로 변환됩니다. 
- 변환된 열은 df['datetime'].dt.year, df['datetime'].dt.dayofweek 등 dt 접근자로 연도, 요일 등 다양한 datetime 연산이 가능합니다.


# (작업중) scikit-learn 문서를 db에 추가 

### 🧭 [Scikit-learn 문서를 LangChain Document 객체로 로드]

</br>

$$
\text{DirectoryLoader} 
\xrightarrow{\text{경로 탐색 및 패턴 매칭 (``*.rst``, ``**/*.rst``)}} 
\text{UnstructuredRSTLoader} 
\xrightarrow{\text{Sphinx 기반 문서의 마크업 제거 및 텍스트 추출}} 
\text{Document 객체}
$$

$$
\text{DirectoryLoader} 
\xrightarrow{\text{경로 탐색 및 패턴 매칭 (``*.md``, ``**/*.md``)}} 
\text{UnstructuredFileLoader} 
\xrightarrow{\text{Markdown 문서의 본문 파싱 및 텍스트 정제}} 
\text{Document 객체}
$$

---

- `DirectoryLoader`는 **Scikit-learn 공식 문서 저장 경로 (`scikit-learn/doc/`)** 내부를 재귀적으로 탐색하며, `.rst` 및 `.md` 파일을 모두 찾아 각 경로를 개별 파일 로더로 전달합니다.

- `.rst` 파일의 경우,  
  **`UnstructuredRSTLoader`**가 Sphinx 스타일 문법(`.. include::`, `:class:`, `:ref:` 등)을 인식해 가능한 한 순수 텍스트를 추출합니다.  
  단, 로컬 `.rst` 파일은 Sphinx 빌드가 적용되지 않기 때문에 수식이나 include 내용 일부가 누락될 수 있습니다.

- `.md` 파일의 경우,  
  **`UnstructuredFileLoader`**가 일반 마크다운 구문(`##`, `-`, 코드 블록 등)을 정제하여 텍스트 기반의 `Document` 객체로 변환합니다.

- 이렇게 생성된 `Document` 객체는 이후 **텍스트 분할기(RecursiveCharacterTextSplitter)**를 통해
RAG 시스템에 적합한 형태(청크 단위)로 가공됩니다.


In [27]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredFileLoader
from langchain_community.document_loaders.rst import UnstructuredRSTLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings  # 또는 HuggingFaceEmbeddings 등

# --- 0. 경로 설정 ---
PANDAS_DB_PATH = r"C:\Users\Hopedom\Documents\VectorDBs\pandas_vectorstore"
SKLEARN_DOC_PATH = r"C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc"
MERGED_DB_PATH = r"C:\Users\Hopedom\Documents\VectorDBs\pandas_sklearn_vectorstore"

print("🔄 Scikit-learn 문서 로드 및 기존 Pandas DB 병합 시작...")

# --- 1. Scikit-learn 문서 로드 ---
sklearn_docs_list = []
seen_sources = set()

# .rst 파일 로드
for pattern in ["*.rst", "**/*.rst"]:
    loader = DirectoryLoader(
        path=SKLEARN_DOC_PATH,
        glob=pattern,
        loader_cls=UnstructuredRSTLoader,
        loader_kwargs={"autodetect_encoding": True}
    )
    for doc in loader.load():
        src = doc.metadata.get("source")
        if src and src not in seen_sources:
            sklearn_docs_list.append(doc)
            seen_sources.add(src)

# .md 파일 로드
for pattern in ["*.md", "**/*.md"]:
    loader = DirectoryLoader(
        path=SKLEARN_DOC_PATH,
        glob=pattern,
        loader_cls=UnstructuredFileLoader,
        loader_kwargs={"autodetect_encoding": True}
    )
    for doc in loader.load():
        src = doc.metadata.get("source")
        if src and src not in seen_sources:
            sklearn_docs_list.append(doc)
            seen_sources.add(src)

print(f"✅ Scikit-learn 문서 로드 완료. 총 {len(sklearn_docs_list)}개 파일")

🔄 Scikit-learn 문서 로드 및 기존 Pandas DB 병합 시작...


[WARNING] Could not load include file maintainers.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 52 column 1
[WARNING] Could not load include file documentation_team.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 70 column 1
[WARNING] Could not load include file contributor_experience_team.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 79 column 1
[WARNING] Could not load include file communication_team.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 87 column 1
[WARNING] Could not load include file maintainers_emeritus.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 99 column 1
[WARNING] Could not load include file communication_team_emeritus.rst at C:\Users\Hopedom\Documents\DS5-LangChain\scikit-learn\doc\about.rst line 107 column 1
[WARNING] Could not load include file contributor_experience_team_emeritus.rst at C:\Users\H

✅ Scikit-learn 문서 로드 완료. 총 192개 파일


In [ ]:
# --- 2. 청크 분할 ---
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
sklearn_docs_chunked = splitter.split_documents(sklearn_docs_list)

print(f"🧩 청크 분할 완료: {len(sklearn_docs_chunked)}개 청크")

# --- 3. 기존 Pandas 벡터스토어 로드 ---
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 또는 HuggingFaceEmbeddings()

print(f"📂 기존 Pandas DB 로드 중: {PANDAS_DB_PATH}")
pandas_vectorstore = FAISS.load_local(
    PANDAS_DB_PATH, 
    embeddings,
    allow_dangerous_deserialization=True  # 필수 (LangChain 0.3.x)
)

# --- 4. Scikit-learn 문서 임베딩 후 추가 ---
print("➕ Scikit-learn 문서 임베딩 및 DB 추가 중...")
pandas_vectorstore.add_documents(sklearn_docs_chunked)

# --- 5. 통합 DB 저장 ---
pandas_vectorstore.save_local(MERGED_DB_PATH)

print(f"🎉 병합 완료: 통합 Vector DB가 {MERGED_DB_PATH} 에 저장되었습니다.")
